En este archivo se mostraran los resúmenes de las métricas de rendimiento que cada modelo ha obtenido en los procesos anteriores. Además, se implementarán distintos "ensemble" de los modelos, para obtener resultados globales con todos los modelos entrenados. 

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
METHODS = ['knn', 'svm', 'naive_bayes', 'random_forest']
SETS = [
    'norm', 'norm_PCA80', 'norm_PCA95',
    'original', 'original_PCA80', 'original_PCA95',
    'stand', 'stand_PCA80', 'stand_PCA95'
]
METRICS_NAMES = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1_Score', 'FNR', 'FPR', 'AUC']
CLASSES = ['0', '1', '2']

In [ ]:
# Creación de directorios necesarios
CWD = os.getcwd()
RESULTS_PATH = os.path.join(CWD, "results")
FIGURES_PATH = os.path.join(CWD, "figures")

METRICS_PATH = os.path.join(CWD, "metrics") # Este se creó en eval.ipynb

try:
    os.mkdir(RESULTS_PATH)
    os.mkdir(FIGURES_PATH)
except FileExistsError:
    pass

## Función

In [ ]:
def calcula_media_desv_tipica(method: str, set: str):

    # Cargamos las métricas del modelo y conjunto correspondientes
    full_path = os.path.join(METRICS_PATH, f"{method}_{set}_metrics.csv")
    metrics = pd.read_csv(full_path, index_col="fold")
    
    # Calculamos media y desv. típica de cada columna
    mean = metrics.mean().to_numpy()
    std = metrics.std().to_numpy()

    results = [f"{set}"]
    for i in range(0, mean.shape[0]):
        results.append(f"{mean[i]:.3f} $\pm$ {std[i]:.3f}")

    return results

<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:13: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_48048/767665287.py:13: SyntaxWarning: invalid escape sequence '\p'
  results.append(f"{mean[i]:.3f} $\pm$ {std[i]:.3f}")


## Generar csv

In [ ]:
f1_summary_path_tex_ori = os.path.join(RESULTS_PATH, f"ori_f1_table.tex")
f1_summary_path_tex_norm = os.path.join(RESULTS_PATH, f"norm_f1_table.tex")
f1_summary_path_tex_stand = os.path.join(RESULTS_PATH, f"stand_f1_table.tex")

f1_summary_path_csv = os.path.join(RESULTS_PATH, f"f1_table.csv")

# Tabla para almacenar los f1-score de cada modelo en cada conjunto de datos
f1_summary_table = []

for method in METHODS:
    
    # Generamos la tabla con las métricas del método correspondiente
    tabla_metodo = []
    for set in SETS:
        fila = calcula_media_desv_tipica(method, set)
        tabla_metodo.append(fila)

    # Generamos un dataframe a partir de dicha tabla
    df = pd.DataFrame(tabla_metodo, columns=["Set", *METRICS_NAMES])
    
    # Guardamos los resultados de la tabla en formato latex y csv
    csv_path = os.path.join(RESULTS_PATH, f"{method}.csv")
    tex_path = os.path.join(RESULTS_PATH, f"{method}.tex")
    df.to_csv(csv_path, index=False)
    df.to_latex(tex_path, index=False, caption=f"Resultados para {method}")

    f1_values = df["F1_Score"].to_list()
    f1_summary_table.append(f1_values)

f1_summary_table_df = pd.DataFrame(f1_summary_table, columns=SETS, index=METHODS)

# Guardamos la tabla con los f1-scores en formatos latex y csv
f1_summary_table_df.to_csv(f1_summary_path_csv)

f1_summary_table_df[['norm', 'norm_PCA80', 'norm_PCA95']] \
    .to_latex(f1_summary_path_tex_norm, caption="Resumen F1-Score por Método y Datasets normalizados")

f1_summary_table_df[['original', 'original_PCA80', 'original_PCA95']] \
    .to_latex(f1_summary_path_tex_ori, caption="Resumen F1-Score por Método y Dataset originales")

f1_summary_table_df[['stand', 'stand_PCA80', 'stand_PCA95']] \
    .to_latex(f1_summary_path_tex_stand, caption="Resumen F1-Score por Método y Dataset estandarizados")
            

In [ ]:
plot_data = []

for method in METHODS:
    for dataset in SETS:
        full_path = os.path.join(METRICS_PATH, f"{method}_{dataset}_metrics.csv")
        
        if os.path.exists(full_path):
            metrics = pd.read_csv(full_path, index_col="fold")
            means = metrics.mean()
            
            entry = {
                'Method': method,
                'Set': dataset,
                'FP': means.get('FPR', 0), 
                'FN': means.get('FNR', 0), 
                'PR': means.get('Precision', 0),
                'RC': means.get('Sensitivity', 0),
                'ACC': means.get('Accuracy', 0),
                'Fm': means.get('F1_Score', 0)
            }
            plot_data.append(entry)

df_plot = pd.DataFrame(plot_data)

# 2. Función para dibujar las líneas
def plot_lines_comparison(method_name, df_method, metric1, metric2, label1, label2, title, filename):
    plt.figure(figsize=(12, 6))
    
    # Eje X: Los datasets en el orden definido en SETS
    x_values = df_method['Set']
    y1_values = df_method[metric1]
    y2_values = df_method[metric2]
    
    # Dibujamos las dos líneas
    plt.plot(x_values, y1_values, marker='o', linestyle='-', linewidth=2, label=label1, color='blue')
    plt.plot(x_values, y2_values, marker='s', linestyle='--', linewidth=2, label=label2, color='orange')
    
    plt.title(f"{title} - {method_name}", fontsize=14)
    plt.xlabel("Conjunto de Datos", fontsize=12)
    plt.ylabel("Valor Métrica", fontsize=12)
    plt.ylim(-0.05, 1.05) # Fijamos el eje Y entre 0 y 1 para ver mejor las diferencias
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    
    # Rotamos las etiquetas del eje X para que se lean bien
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    save_path = os.path.join(FIGURES_PATH, filename)
    plt.savefig(save_path)
    plt.close()

# 3. Bucle principal
for method in METHODS:
    # Filtramos los datos solo para el método actual
    # Reindexamos por SETS para asegurar que el eje X siga el orden lógico (Norm -> Norm_PCA...)
    # y no el orden alfabético aleatorio.
    df_method = df_plot[df_plot['Method'] == method].set_index('Set').reindex(SETS).reset_index()
    
    # a) ACC contra Fm
    plot_lines_comparison(
        method, df_method, 
        'ACC', 'Fm', 
        'Accuracy', 'F1-Score', 
        'Comparativa Accuracy vs F1-Score', 
        f"ACC_vs_Fm_{method}.png"
    )
    
    # b) FN contra FP
    plot_lines_comparison(
        method, df_method, 
        'FN', 'FP', 
        'False Negative Rate', 'False Positive Rate', 
        'Comparativa FN vs FP', 
        f"FN_vs_FP_{method}.png"
    )
    
    # c) PR contra RC
    plot_lines_comparison(
        method, df_method, 
        'PR', 'RC', 
        'Precision', 'Recall', 
        'Comparativa Precision vs Recall', 
        f"PR_vs_RC_{method}.png"
    )

# Ensemble

In [ ]:
def obtener_probabilidad(model_name, fold, set):
    """
    Obtienes el dataframe de probabilidades del modelo, fold y set especificado

    Args:
        model_name (str): Nombre del modelo (e.g., 'knn').
        fold (int): Número de la partición (e.g., 1).
        set_name (str): Versión del conjunto de datos (e.g., 'norm').

    Returns:
        pd.DataFrame: DataFrame con los resultados, o None si el archivo no existe.
    """
    # Obtenemos el path del csv y lo cargamos
    file_name = f"pred_{fold}_{set}_{model_name}.csv"
    full_path = os.path.join("predictions", file_name)

    predictions_df = pd.read_csv(full_path)

    return predictions_df

### Votación

In [ ]:
try:
    os.mkdir(path="ensemble_vote")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        # Añadimos los votos
        for df in prediction_dfs:
            predicted_classes = df[CLASSES].idxmax(axis=1)

            for index, clase in predicted_classes.items():
                df_median.loc[index, clase] += 1

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"votes_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_vote", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_vote/votes_fold1_norm.csv con 4 modelos.
Generado: ensemble_vote/votes_fold2_norm.csv con 4 modelos.
Generado: ensemble_vote/votes_fold3_norm.csv con 4 modelos.
Generado: ensemble_vote/votes_fold4_norm.csv con 4 modelos.
Generado: ensemble_vote/votes_fold5_norm.csv con 4 modelos.
Generado: ensemble_vote/votes_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote/votes_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote/votes_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote/votes_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote/votes_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_vote/votes_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote/votes_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote/votes_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote/votes_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote/votes_fold5_norm_PCA95.csv con 4 modelos.
Generado: ensemble_vote/votes_fold1_orig

### Media

In [ ]:
try:
    os.mkdir(path="ensemble_avg")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        number_methods = len(METHODS)

        # Sumamos todos los DataFrames
        for df in prediction_dfs:
            df_median = df_median.add(df, fill_value=0)

        # Calculamos la media y redondeamos
        df_median = df_median / number_methods
        df_median[CLASSES] = df_median[CLASSES].round(3)   

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"avg_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_avg", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_avg/avg_fold1_norm.csv con 4 modelos.
Generado: ensemble_avg/avg_fold2_norm.csv con 4 modelos.
Generado: ensemble_avg/avg_fold3_norm.csv con 4 modelos.
Generado: ensemble_avg/avg_fold4_norm.csv con 4 modelos.
Generado: ensemble_avg/avg_fold5_norm.csv con 4 modelos.
Generado: ensemble_avg/avg_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg/avg_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg/avg_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg/avg_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg/avg_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_avg/avg_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg/avg_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg/avg_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg/avg_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg/avg_fold5_norm_PCA95.csv con 4 modelos.
Generado: ensemble_avg/avg_fold1_original.csv con 4 modelos.
Generado: ensemble_avg/a

### Mediana

In [ ]:
try:
    os.mkdir(path="ensemble_median")
except FileExistsError:
    pass

for set in SETS:
    for fold in range(1,6):
        
        prediction_dfs = []

        for method in METHODS:
            # Metemos en una lista todos los dfs de probabilidades para ese fold y set
            probabilities = obtener_probabilidad(method, fold, set)
            if not probabilities.empty:
                prediction_dfs.append(probabilities)

        # Creamos el DataFrame base
        df_median = pd.DataFrame(0, index=prediction_dfs[0].index, columns=CLASSES, dtype=int)

        # Concatemos los DataFrames para que sea más sencillo el cálculo de la mediana
        df_all_preds = pd.concat(prediction_dfs)

        # Agrupamos por el índice original (y calculamos la mediana
        df_median = df_all_preds.groupby(level=0)[CLASSES].median()
        df_median[CLASSES] = df_median[CLASSES].round(3)

        # Añadimos la columna que recoja la decisión del Ensemble
        df_median['Ensemble_Decision'] = df_median[CLASSES].idxmax(axis=1)

        # Guardamos en csv el DataFrame de votos
        file_name = f"median_fold{fold}_{set}.csv"
        full_path = os.path.join("ensemble_median", file_name)
            
        df_median.to_csv(full_path, index=True, index_label='Index')
        print(f"Generado: {full_path} con {len(prediction_dfs)} modelos.")        

Generado: ensemble_median/median_fold1_norm.csv con 4 modelos.
Generado: ensemble_median/median_fold2_norm.csv con 4 modelos.
Generado: ensemble_median/median_fold3_norm.csv con 4 modelos.
Generado: ensemble_median/median_fold4_norm.csv con 4 modelos.
Generado: ensemble_median/median_fold5_norm.csv con 4 modelos.
Generado: ensemble_median/median_fold1_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median/median_fold2_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median/median_fold3_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median/median_fold4_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median/median_fold5_norm_PCA80.csv con 4 modelos.
Generado: ensemble_median/median_fold1_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median/median_fold2_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median/median_fold3_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median/median_fold4_norm_PCA95.csv con 4 modelos.
Generado: ensemble_median/median_fold5_norm_PCA95.csv con 4 mode